In [40]:
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup
from bs4 import ResultSet, Tag
import datetime
import requests
import pytz
import time
import re
import os

import pandas as pd

In [2]:
LIMIT = 500

In [3]:
movie_title = input('Write a movie title: ')
movie_title

'what if'

In [50]:
def get_reviews(title: str):
    title_query = title.replace(' ', '%20').lower().strip()
    url = f"https://www.imdb.com/find/?q={title_query}&ref_=nv_sr_sm"
    options = webdriver.ChromeOptions()
    chrome_install = ChromeDriverManager().install()

    folder = os.path.dirname(chrome_install)
    chromedriver_path = os.path.join(folder, "chromedriver.exe")

    service = ChromeService(chromedriver_path)
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)

    time.sleep(2)
    first_movie = driver.find_elements(By.CLASS_NAME, 'ipc-metadata-list-summary-item__t')[0]
    first_movie.click()

    time.sleep(2)
    user_review_area = driver.find_element(By.CSS_SELECTOR, '[data-testid="UserReviews"]')
    ActionChains(driver).scroll_to_element(user_review_area).perform()
    user_review_button = user_review_area.find_elements(By.CLASS_NAME, "ipc-title__text")[0]
    user_review_button.click()
    time.sleep(2)

    reviews_count: str = driver.find_element(By.CSS_SELECTOR, '[data-testid="tturv-total-reviews"]').get_attribute('innerHTML')
    reviews_count = reviews_count.replace(" reviews", '')
    reviews_count = int(reviews_count)
    print("Total reviews:", reviews_count)


    pagination = driver.find_element(By.CSS_SELECTOR, '[data-testid="tturv-pagination"]')
    ActionChains(driver).scroll_to_element(pagination).perform()
    pagination.find_elements(By.TAG_NAME, 'span')[-1].click()
    time.sleep(30)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    reviews = soup.find_all('h3')
    reviews = [i.get_text(strip=True) for i in reviews]

    driver.quit()

    if (len(reviews) - 6) < reviews_count:
        get_reviews(title)
    else:
        return reviews

In [51]:
reviews = get_reviews(movie_title)

Total reviews: 506


In [39]:
print("Total data scraped:", len(reviews))
reviews

Total data scraped: 512


['I liked the first season better',
 'S1=Amazing S2=Meh',
 'Cool series with interesting twists n tales.',
 'Great scenarios, but varies between episode',
 'Doctor Strange Episode...Wow',
 'Too much Captain Carter!',
 'What If...I Love It And Need More?',
 'A lot of wasted potential. A couple of good episodes, though.',
 'People need to review the entire show instead of only 1 episode',
 'Marvel, please take your cue from the original series!',
 'Be patient!',
 'Season 1 had some merit; Season 2 is much worse',
 'What if season 2 is all about Peggy Carter?',
 'Good animation, poor pacing, quite disappointing really.',
 'Keep watching, it really takes of from episode 3',
 'Not that bad',
 'Uninteresting finale and overuse of Peggy Carter',
 "It's promising",
 'Rated the first one with 6stars, increasing 1 star after each episode',
 'Love it!',
 'Premise is interesting.',
 'What if...what if is the best animated Marvel series ever?',
 '4 Eps so far... Surprisingly Limp and Wasted Potenti

In [42]:
df = pd.DataFrame(reviews[:-6], columns=['reviews'])
df.head()

,reviews
0,I liked the first season better
1,S1=Amazing S2=Meh
2,Cool series with interesting twists n tales.
3,"Great scenarios, but varies between episode"
4,Doctor Strange Episode...Wow
